# Data Aggregation and Group Operations

Todos aquellos métodos que nos permiten seleccionar y agrupar datos para operar con todos juntos a la vez

# Índice

1. GroupBy Mechanics
    
    1.1 Iterating Over Groups
    
    1.2 Grouping with Dicts and Series
    
    1.3 Grouping with Functions
    
    1.4 Grouping by Index Levels
    
    
2. Data Aggregation

    2.1 Column-Wise Aggregation
    
    
3. Apply Function


4. Pivot Tables and Cross-Tabulation

# Load Data

In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

# 1. GroupBy Mechanics

Esta funcionalidad permite hacer agregaciones y también hacer operaciones en base a estas agregaciones como el cálculo de sumas, medias, desviaciones típicas, etc.

In [3]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'], 'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5), 'data2' : np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,0.540382,0.900821
1,a,two,-0.189534,0.470286
2,b,one,-0.007663,0.410027
3,b,two,-0.809625,-0.308014
4,a,one,-0.149492,-0.909523


In [10]:
df.groupby(["key1"]).mean()

,data1,data2
key1,,
a,0.067119,0.153861
b,-0.408644,0.051007


#### Podemos seleccionar una columna en específico

In [12]:
df["data1"].groupby(df["key1"]).mean()

key1
a    0.067119
b   -0.408644
Name: data1, dtype: float64

In [39]:
df.groupby("key1")[["data1"]].mean()

,data1
key1,
a,0.067119
b,-0.408644


#### Podemos seleccionar varias columnas

In [40]:
df["data2"].groupby([df["key1"], df["key2"]]).mean()

key1  key2
a     one    -0.004351
      two     0.470286
b     one     0.410027
      two    -0.308014
Name: data2, dtype: float64

In [41]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    one  -0.004351
     two   0.470286
b    one   0.410027
     two  -0.308014

#### Podemos crear un objeto

In [13]:
grouped = df["data1"].groupby(df["key1"])

grouped

In [14]:
grouped.mean()

key1
a    0.067119
b   -0.408644
Name: data1, dtype: float64

#### Podemos pasar múltiples groupby

In [16]:
df["data1"].groupby([df["key1"], df["key2"]]).mean()

key1  key2
a     one     0.195445
      two    -0.189534
b     one    -0.007663
      two    -0.809625
Name: data1, dtype: float64

#### Podemos eliminar niveles con unstack y convertirlo en DataFrame

In [19]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()

means.unstack()

key2,one,two
key1,,
a,0.195445,-0.189534
b,-0.007663,-0.809625


#### Podemos medir el número de veces que se repite la agrupación con size

In [25]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## 1.1 Iterating Over Groups
Groupby soporta iteraciones

#### Single key

In [31]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)
    print("")

a
  key1 key2     data1     data2
0    a  one  0.540382  0.900821
1    a  two -0.189534  0.470286
4    a  one -0.149492 -0.909523

b
  key1 key2     data1     data2
2    b  one -0.007663  0.410027
3    b  two -0.809625 -0.308014



#### Multiple keys:

In [32]:
for (k1,k2), group in df.groupby(["key1", "key2"]):
    print((k1,k2))
    print(group)
    print("")

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.540382  0.900821
4    a  one -0.149492 -0.909523

('a', 'two')
  key1 key2     data1     data2
1    a  two -0.189534  0.470286

('b', 'one')
  key1 key2     data1     data2
2    b  one -0.007663  0.410027

('b', 'two')
  key1 key2     data1     data2
3    b  two -0.809625 -0.308014



#### Computing a dict of the data

In [33]:
pieces = dict(list(df.groupby("key1")))

pieces

{'a':   key1 key2     data1     data2
 0    a  one  0.540382  0.900821
 1    a  two -0.189534  0.470286
 4    a  one -0.149492 -0.909523,
 'b':   key1 key2     data1     data2
 2    b  one -0.007663  0.410027
 3    b  two -0.809625 -0.308014}

In [34]:
pieces["b"]

,key1,key2,data1,data2
2,b,one,-0.007663,0.410027
3,b,two,-0.809625,-0.308014


## 1.2 Grouping with Dicts and Series

In [42]:
people = pd.DataFrame(np.random.randn(5, 5), columns=['a', 'b', 'c', 'd', 'e'], 
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people

,a,b,c,d,e
Joe,0.489082,-0.518234,-0.946116,1.542775,-0.422900
Steve,0.436923,-0.863873,0.960337,-1.761184,2.095261
Wes,-0.861539,1.399101,0.949666,-0.462619,1.434022
Jim,0.204128,0.356342,-1.249076,1.132067,-0.798306
Travis,-0.614697,-0.411239,0.123013,-2.889617,-0.315782


In [43]:
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values

people

,a,b,c,d,e
Joe,0.489082,-0.518234,-0.946116,1.542775,-0.422900
Steve,0.436923,-0.863873,0.960337,-1.761184,2.095261
Wes,-0.861539,NaN,NaN,-0.462619,1.434022
Jim,0.204128,0.356342,-1.249076,1.132067,-0.798306
Travis,-0.614697,-0.411239,0.123013,-2.889617,-0.315782


In [44]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [46]:
by_column = people.groupby(mapping, axis = 1)

by_column.sum()

,blue,red
Joe,0.596659,-0.452052
Steve,-0.800847,1.668310
Wes,-0.462619,0.572483
Jim,-0.117009,-0.237836
Travis,-2.766604,-1.341718


## 1.3 Grouping with Functions

In [49]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.168330,-0.161892,-2.195192,2.212222,0.212816
5,0.436923,-0.863873,0.960337,-1.761184,2.095261
6,-0.614697,-0.411239,0.123013,-2.889617,-0.315782


In [50]:
key_list = ['one', 'one', 'one', 'two', 'two'] # Cada nombre de los index se asocia a un key list

people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.861539 -0.518234 -0.946116 -0.462619 -0.422900
  two  0.204128  0.356342 -1.249076  1.132067 -0.798306
5 one  0.436923 -0.863873  0.960337 -1.761184  2.095261
6 two -0.614697 -0.411239  0.123013 -2.889617 -0.315782

## 1.4 Grouping by Index Levels

In [51]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], [1, 3, 5, 1, 3]], 
                                    names=['cty', 'tenor'])

columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [53]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)

hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.223211  1.015798  0.251998  0.896201 -0.791786
1     -1.356967 -0.722561  0.858615 -1.723005  0.483099
2      0.865277  1.163820  0.765556 -0.077723 -0.052425
3      0.164045  0.003959  0.200392 -0.568729 -0.297269

In [54]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [131]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://i.gyazo.com/ae7ce4bbf934e3ff883bdb62c2a9f5d1.png", width = 400)

# 2. Data Aggregation
Cualquier transformación que produzca un escalar nuevo

In [55]:
df

,key1,key2,data1,data2
0,a,one,0.540382,0.900821
1,a,two,-0.189534,0.470286
2,b,one,-0.007663,0.410027
3,b,two,-0.809625,-0.308014
4,a,one,-0.149492,-0.909523


#### Agregaciones con funciones predeterminadas (Series method)

In [56]:
grouped = df.groupby('key1')

grouped['data1'].quantile(0.9)

key1
a    0.402407
b   -0.087859
Name: data1, dtype: float64

#### Agregacioens con funciones propias

In [57]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.729915,1.810344
b,0.801961,0.718042


## 2.1 Column Wise Aggregation

In [59]:
tips = pd.read_csv("DataBases/seaborn-data-master/tips.csv")

tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [60]:
tips["tips_pct"] = tips["tip"] / tips["total_bill"]

tips.head()

,total_bill,tip,sex,smoker,day,time,size,tips_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [64]:
grouped = tips.groupby(["day", "smoker"])

grouped_pct = grouped["tips_pct"]

In [65]:
grouped_pct.mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tips_pct, dtype: float64

#### Podemos hacer agregaciones simples con funciones

In [67]:
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tips_pct, dtype: float64

#### Podemos hacer agregaciones con múltiples funciones, tanto preestablecidas como customizadas

In [74]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

#### Si pasamos una lista de tuplas (name, function), el primer elemento de cada tupla se considera la columna del DataFrame

In [75]:
grouped_pct.agg([("foo", "mean"), ("bar", peak_to_peak)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.067349
     Yes     0.174783  0.159925
Sat  No      0.158048  0.235193
     Yes     0.147906  0.290095
Sun  No      0.160113  0.193226
     Yes     0.187250  0.644685
Thur No      0.160298  0.193350
     Yes     0.163863  0.151240

#### Podemos crear una lista de funciones para pasarla posteriormente

In [80]:
functions = ["count", "mean", "max", peak_to_peak]

grouped.agg(functions)

total_bill                                  tip                   \
                 count       mean    max peak_to_peak count      mean    max   
day  smoker                                                                    
Fri  No              4  18.420000  22.75        10.29     4  2.812500   3.50   
     Yes            15  16.813333  40.17        34.42    15  2.714000   4.73   
Sat  No             45  19.661778  48.33        41.08    45  3.102889   9.00   
     Yes            42  21.276667  50.81        47.74    42  2.875476  10.00   
Sun  No             57  20.506667  48.17        39.40    57  3.167895   6.00   
     Yes            19  24.120000  45.35        38.10    19  3.516842   6.50   
Thur No             45  17.113111  41.19        33.68    45  2.673778   6.70   
     Yes            17  19.190588  43.11        32.77    17  3.030000   5.00   

                          size                            tips_pct            \
            peak_to_peak count      mean max peak_to_peak    count      mean   
day  smoker                                                                    
Fri  No             2.00     4  2.250000   3            1        4  0.151650   
     Yes            3.73    15  2.066667   4            3       15  0.174783   
Sat  No             8.00    45  2.555556   4            3       45  0.158048   
     Yes            9.00    42  2.476190   5            4       42  0.147906   
Sun  No             4.99    57  2.929825   6            4       57  0.160113   
     Yes            5.00    19  2.578947   5            3       19  0.187250   
Thur No             5.45    45  2.488889   6            5       45  0.160298   
     Yes            3.00    17  2.352941   4            2       17  0.163863   

                                    
                  max peak_to_peak  
day  smoker                         
Fri  No      0.187735     0.067349  
     Yes     0.263480     0.159925  
Sat  No      0.291990     0.235193  
     Yes     0.325733     0.290095  
Sun  No      0.252672     0.193226  
     Yes     0.710345     0.644685  
Thur No      0.266312     0.193350  
     Yes     0.241255     0.151240

In [87]:
grouped[["tips_pct", "total_bill"]].agg(functions)

tips_pct                                  total_bill             \
               count      mean       max peak_to_peak      count       mean   
day  smoker                                                                   
Fri  No            4  0.151650  0.187735     0.067349          4  18.420000   
     Yes          15  0.174783  0.263480     0.159925         15  16.813333   
Sat  No           45  0.158048  0.291990     0.235193         45  19.661778   
     Yes          42  0.147906  0.325733     0.290095         42  21.276667   
Sun  No           57  0.160113  0.252672     0.193226         57  20.506667   
     Yes          19  0.187250  0.710345     0.644685         19  24.120000   
Thur No           45  0.160298  0.266312     0.193350         45  17.113111   
     Yes          17  0.163863  0.241255     0.151240         17  19.190588   

                                 
               max peak_to_peak  
day  smoker                      
Fri  No      22.75        10.29  
     Yes     40.17        34.42  
Sat  No      48.33        41.08  
     Yes     50.81        47.74  
Sun  No      48.17        39.40  
     Yes     45.35        38.10  
Thur No      41.19        33.68  
     Yes     43.11        32.77

In [88]:
grouped["tips_pct"].agg(functions)

count      mean       max  peak_to_peak
day  smoker                                         
Fri  No          4  0.151650  0.187735      0.067349
     Yes        15  0.174783  0.263480      0.159925
Sat  No         45  0.158048  0.291990      0.235193
     Yes        42  0.147906  0.325733      0.290095
Sun  No         57  0.160113  0.252672      0.193226
     Yes        19  0.187250  0.710345      0.644685
Thur No         45  0.160298  0.266312      0.193350
     Yes        17  0.163863  0.241255      0.151240

#### Podemos pasar una lista de tuplas con nombres customizados

In [92]:
ftuples = [('custom_1_MEAN', 'mean'), ('custom_2_VAR', np.var)]

grouped[['tips_pct', 'total_bill']].agg(ftuples)

tips_pct                 total_bill             
            custom_1_MEAN custom_2_VAR custom_1_MEAN custom_2_VAR
day  smoker                                                      
Fri  No          0.151650     0.000791     18.420000    25.596333
     Yes         0.174783     0.002631     16.813333    82.562438
Sat  No          0.158048     0.001581     19.661778    79.908965
     Yes         0.147906     0.003767     21.276667   101.387535
Sun  No          0.160113     0.001793     20.506667    66.099980
     Yes         0.187250     0.023757     24.120000   109.046044
Thur No          0.160298     0.001503     17.113111    59.625081
     Yes         0.163863     0.001551     19.190588    69.808518

#### Podemos aplicar diferentes funciones a las columnas

In [93]:
grouped.agg({"tip": np.max, "size": "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [96]:
grouped.agg({"tip": ['min', 'max', 'mean', 'std'], 'size' : 'sum'})

tip                            size
              min    max      mean       std  sum
day  smoker                                      
Fri  No      1.50   3.50  2.812500  0.898494    9
     Yes     1.00   4.73  2.714000  1.077668   31
Sat  No      1.00   9.00  3.102889  1.642088  115
     Yes     1.00  10.00  2.875476  1.630580  104
Sun  No      1.01   6.00  3.167895  1.224785  167
     Yes     1.50   6.50  3.516842  1.261151   49
Thur No      1.25   6.70  2.673778  1.282964  112
     Yes     2.00   5.00  3.030000  1.113491   40

# 3. Apply Function
Apply divide el objeto para manipularlo por partes, invoca la función que queremos realizar y después concatena todas las partes

In [99]:
Image(url= "https://jakevdp.github.io/figures/split-apply-combine.svg", width = 500)

In [100]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tips_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


Por ejemplo, vamos a definir una función que nos devuelva los top n "tips_pct"

In [101]:
def top (tips, n, column = "tips_pct"):
    return tips.sort_values(by=column)[-n:]

In [102]:
top(tips, n = 5)

,total_bill,tip,sex,smoker,day,time,size,tips_pct
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


#### Podemos utilizar el método "apply" para pasarla en el dataframe

Buscamos el top 5 "tips_pct" según el groupby "smoker"

In [103]:
tips.groupby("smoker").apply(top, 5)

total_bill   tip     sex smoker   day    time  size  tips_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

#### Podemos eliminar las "group keys", es decir, los niveles jerárquicos

In [104]:
tips.groupby("smoker", group_keys = False).apply(top, 5)

,total_bill,tip,sex,smoker,day,time,size,tips_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


#### Podemos seleccionar hacer agrupaciones "groupby" de más de una columna

Buscamos el top 1 de cada día, dividido por fumador o no fumador en la columna seleccionada por la función top ("tips_pct")

In [107]:
tips.groupby(["smoker", "day"]).apply(top, 1)

total_bill   tip     sex smoker   day    time  size  tips_pct
smoker day                                                                    
No     Fri  223       15.98  3.00  Female     No   Fri   Lunch     3  0.187735
       Sat  232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
       Sun  51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       Thur 149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
Yes    Fri  93        16.32  4.30  Female    Yes   Fri  Dinner     2  0.263480
       Sat  67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       Sun  172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345
       Thur 194       16.58  4.00    Male    Yes  Thur   Lunch     2  0.241255

#### Aunque tengamos la función ya creada, podemos cambiar argumentos de la función. Por ejemplo, "column"

Ahora vamos a buscar el top 1 de cada día, dividido por fumador o no fumador en la columna que vamos a seleccionar ahora ("total_bill")

In [108]:
tips.groupby(["smoker", "day"]).apply(top, 1, column = "total_bill")

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   

                 tips_pct  
smoker day                 
No     Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799  
       Thur 142  0.121389  
Yes    Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
       Thur 197  0.115982

#### Podemos realizar ciertas operaciones estadísticas con el método groupby de por medio

In [116]:
tips.groupby(["smoker", "day"])["total_bill"].describe()

count       mean        std    min     25%     50%      75%  \
smoker day                                                                 
No     Fri     4.0  18.420000   5.059282  12.46  15.100  19.235  22.5550   
       Sat    45.0  19.661778   8.939181   7.25  14.730  17.820  20.6500   
       Sun    57.0  20.506667   8.130189   8.77  14.780  18.430  25.0000   
       Thur   45.0  17.113111   7.721728   7.51  11.690  15.950  20.2700   
Yes    Fri    15.0  16.813333   9.086388   5.75  11.690  13.420  18.6650   
       Sat    42.0  21.276667  10.069138   3.07  13.405  20.390  26.7925   
       Sun    19.0  24.120000  10.442511   7.25  17.165  23.100  32.3750   
       Thur   17.0  19.190588   8.355149  10.34  13.510  16.470  19.8100   

               max  
smoker day          
No     Fri   22.75  
       Sat   48.33  
       Sun   48.17  
       Thur  41.19  
Yes    Fri   40.17  
       Sat   50.81  
       Sun   45.35  
       Thur  43.11

# 4. Pivot Tables and Cross-Tabulation
Nos permite crear una tabla en la que podemos transformar una o más columnas como filas y viceversa. Siempre devuelve "means" por defecto

In [118]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tips_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [119]:
tips.pivot_table(index = ["day", "smoker"])

size       tip  tips_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

#### Podemos seleccionar qué queremos que aparezca en la tabla, y las columnas que queremos 

In [123]:
tips.pivot_table(['tips_pct', 'size'], index=['day', "smoker"], columns='sex')

size            tips_pct          
sex            Female      Male    Female      Male
day  smoker                                        
Fri  No      2.500000  2.000000  0.165296  0.138005
     Yes     2.000000  2.125000  0.209129  0.144730
Sat  No      2.307692  2.656250  0.147993  0.162132
     Yes     2.200000  2.629630  0.163817  0.139067
Sun  No      3.071429  2.883721  0.165710  0.158291
     Yes     2.500000  2.600000  0.237075  0.173964
Thur No      2.480000  2.500000  0.155971  0.165706
     Yes     2.428571  2.300000  0.163073  0.164417

#### Podemos incluir una fila que contenga el total

In [127]:
tips.pivot_table(['tips_pct', 'size'], index=['day', "smoker"], columns='sex', margins = True)

size                      tips_pct                    
sex            Female      Male       All    Female      Male       All
day  smoker                                                            
Fri  No      2.500000  2.000000  2.250000  0.165296  0.138005  0.151650
     Yes     2.000000  2.125000  2.066667  0.209129  0.144730  0.174783
Sat  No      2.307692  2.656250  2.555556  0.147993  0.162132  0.158048
     Yes     2.200000  2.629630  2.476190  0.163817  0.139067  0.147906
Sun  No      3.071429  2.883721  2.929825  0.165710  0.158291  0.160113
     Yes     2.500000  2.600000  2.578947  0.237075  0.173964  0.187250
Thur No      2.480000  2.500000  2.488889  0.155971  0.165706  0.160298
     Yes     2.428571  2.300000  2.352941  0.163073  0.164417  0.163863
All          2.459770  2.630573  2.569672  0.166491  0.157651  0.160803

#### Podemos pasar funciones

In [128]:
tips.pivot_table(['tips_pct', 'size'], index=['day', "smoker"], columns='sex', margins = True,
                aggfunc = len)

size           tips_pct              
sex         Female Male  All   Female   Male    All
day  smoker                                        
Fri  No          2    2    4      2.0    2.0    4.0
     Yes         7    8   15      7.0    8.0   15.0
Sat  No         13   32   45     13.0   32.0   45.0
     Yes        15   27   42     15.0   27.0   42.0
Sun  No         14   43   57     14.0   43.0   57.0
     Yes         4   15   19      4.0   15.0   19.0
Thur No         25   20   45     25.0   20.0   45.0
     Yes         7   10   17      7.0   10.0   17.0
All             87  157  244     87.0  157.0  244.0

#### Podemos rellenar los NaN value

In [129]:
tips.pivot_table(['tips_pct', 'size'], index=['day', "smoker"], columns='sex', margins = True,
                fill_value = 0)

size                      tips_pct                    
sex            Female      Male       All    Female      Male       All
day  smoker                                                            
Fri  No      2.500000  2.000000  2.250000  0.165296  0.138005  0.151650
     Yes     2.000000  2.125000  2.066667  0.209129  0.144730  0.174783
Sat  No      2.307692  2.656250  2.555556  0.147993  0.162132  0.158048
     Yes     2.200000  2.629630  2.476190  0.163817  0.139067  0.147906
Sun  No      3.071429  2.883721  2.929825  0.165710  0.158291  0.160113
     Yes     2.500000  2.600000  2.578947  0.237075  0.173964  0.187250
Thur No      2.480000  2.500000  2.488889  0.155971  0.165706  0.160298
     Yes     2.428571  2.300000  2.352941  0.163073  0.164417  0.163863
All          2.459770  2.630573  2.569672  0.166491  0.157651  0.160803